# Import bibliotek

In [13]:
import gdal
from osgeo import gdal
import os
import gdal
from osgeo import osr 
import subprocess
import numpy as np

# Zadanie pierwsze

## Wczytanie pliku asc

In [14]:
path="" 
f="puszcza_bukowa_szmaragdowe.asc"
plik=os.path.join(path,f)
print(plik)

puszcza_bukowa_szmaragdowe.asc


## Konwersja pliku asc do tif

In [15]:
outFileName = 'szmaragdowe_geotif.tif'
nmt = gdal.Open(plik)
srs = osr.SpatialReference()
srs.ImportFromEPSG(2180)


band = nmt.GetRasterBand(1) #wczytujemy jako osgeo.gdal.Dataset

arr = band.ReadAsArray() #wczytujemy jako tablicę
[cols, rows] = arr.shape #konwersja do numpy shape
arr_min = arr.min()
arr_max = arr.max()
arr_mean = int(arr.mean())
print(arr_min)
print(arr_max)
print(arr_mean)

driver = gdal.GetDriverByName("GTiff")
outdata = driver.Create(outFileName, rows, cols, 1, gdal.GDT_Float32)
outdata.SetGeoTransform(nmt.GetGeoTransform())#tra geotransform as input
outdata.SetProjection(srs.ExportToWkt())##usatwienie układu współrzędnych 2180
outdata.GetRasterBand(1).WriteArray(arr)
outdata.GetRasterBand(1).SetNoDataValue(-9999)# błedne piksele przyjmą wartość -9999
outdata.FlushCache() # zapis
nmt=None

40.07
93.99
65


## Reklasyfikacja 

In [16]:
# Wczytanie pliku tif
driver = gdal.GetDriverByName('GTiff')
f = gdal.Open('szmaragdowe_geotif.tif')
band = f.GetRasterBand(1)
array = band.ReadAsArray()

area_per_pixel=f.GetGeoTransform()


# Reklasyfikacja
array[np.where( array <= 5 )] = 1
array[np.where((5 < array) & (array <= 10)) ] = 2
array[np.where((10 < array) & (array <= 20)) ] = 3
array[np.where(20 > array)] = 4

print(array)

# Obliczenie powierzchnni poszczególnych klas
for x in np.unique(array):
    tot_num_pixels = np.sum(array == x)
    area = tot_num_pixels * area_per_pixel[1]
    print (x, area/10000, 'ha')



# Zapis pliku
file2 = driver.Create( 'szmaragdowe_geotif_reclass.tif', f.RasterXSize , f.RasterYSize , 1)
file2.GetRasterBand(1).WriteArray(array)

# układ współrzędnych pobierany z pliku źródłowego
proj = f.GetProjection()
georef = f.GetGeoTransform()
file2.SetProjection(proj)
file2.SetGeoTransform(georef)
file2.FlushCache()
array=None

[[56.42 56.36 56.47 ... 56.03 56.17 56.19]
 [56.5  56.45 56.49 ... 56.05 56.36 56.38]
 [56.61 56.61 56.53 ... 56.34 56.49 56.67]
 ...
 [91.69 91.68 91.7  ... 68.04 67.95 67.6 ]
 [91.67 91.68 91.68 ... 68.25 68.   67.71]
 [91.64 91.64 91.63 ... 68.3  68.05 67.8 ]]
40.07 0.0001 ha
40.11 0.0001 ha
40.18 0.0001 ha
40.28 0.0002 ha
40.3 0.0001 ha
40.36 0.0002 ha
40.4 0.0002 ha
40.41 0.0002 ha
40.43 0.0001 ha
40.45 0.0002 ha
40.46 0.0001 ha
40.47 0.0001 ha
40.53 0.0003 ha
40.55 0.0001 ha
40.57 0.0001 ha
40.58 0.0002 ha
40.59 0.0002 ha
40.6 0.0002 ha
40.61 0.0001 ha
40.63 0.0004 ha
40.64 0.0001 ha
40.66 0.0001 ha
40.67 0.0003 ha
40.69 0.0001 ha
40.71 0.0001 ha
40.72 0.0003 ha
40.73 0.0003 ha
40.77 0.0003 ha
40.78 0.0001 ha
40.79 0.0003 ha
40.81 0.0001 ha
40.82 0.0002 ha
40.84 0.0001 ha
40.85 0.0002 ha
40.86 0.0001 ha
40.87 0.0001 ha
40.89 0.0004 ha
40.91 0.0001 ha
40.93 0.0001 ha
40.95 0.0002 ha
40.96 0.0001 ha
40.98 0.0001 ha
40.99 0.0001 ha
41.01 0.0001 ha
41.02 0.0003 ha
41.03 0.0001 ha
41.

# Zadanie drugie

In [17]:
# Układ współrzędnych oraz format 
# Zapytać o co chodzi z regularną siatką punktów
subprocess.run("gdal_translate -a_srs EPSG:32633 -of GPKG szmaragdowe_geotif.tif szmaragdowe_geotif.gpkg")

CompletedProcess(args='gdal_translate -a_srs EPSG:32633 -of GPKG szmaragdowe_geotif.tif szmaragdowe_geotif.gpkg', returncode=1)

In [12]:
# Układ współrzędnych, komenda gdal_contour
# Zapytać o zapis (shp i czemu nie działa w sumie)
subprocess.run("gdal_contour -a_srs EPSG:2180 -a elev szmaragdowe_geotif.tif szmaragdowe_geotif.shp -i 1.0")

CompletedProcess(args='gdal_contour -a_srs EPSG:2180 -a elev szmaragdowe_geotif.tif szmaragdowe_geotif.shp -i 1.0', returncode=1)